In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy.linalg as scla
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

%matplotlib notebook

## Log Model with upper-bound on $\delta$ || FGD


In [9]:
# display parameters

stepper = 50
mult = 20*stepper

kappa_widget = widgets.FloatSlider(
    value=0.0025,
    min=0.00001,
    max= 0.05,
    step=0.00001,
    description='kappa :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f'
)

main_widget = widgets.IntSlider(
    value=500,
    min=10,
    max= 1500,
    step=10,
    description='main :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

beta_widget = widgets.FloatSlider(
    value=1,
    min=0,
    max= 10,
    step=0.1,
    description='beta :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f'
)

K_widget = widgets.IntSlider(
    value=10,
    min=0,
    max= 50,
    step=5,
    description='K :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

B_widget = widgets.IntSlider(
    value=100*K_widget.value,
    min=K_widget.value,
    max= mult*K_widget.value+stepper,
    step=stepper,
    description='B :',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

def options(selections,N_list,Nb_list,eps_list,eps_constant_list,N_opt,N_opt_constant,opt_sched_const,opt_Nb,opt_sched,\
           conv_term,error_term,conv_term_const,error_term_const,Nb_smooth,values):

    for selection in selections:
    
        if selection=='acc vs N at B':
            plt.figure(figsize=(6,4))
            plt.grid()
            plt.title('$\epsilon^*(N,B)$')
            plt.xlabel('# iters')
            plt.ylabel('$\epsilon^*$')
            plt.semilogy(N_list,eps_list)
            plt.semilogy(N_list,eps_constant_list)
            plt.scatter([N_opt],[np.min(eps_list)])
            plt.scatter([N_opt_constant],[np.min(eps_constant_list)])
            plt.legend(['adaptive $\epsilon^*$','fixed $\epsilon^*$'])

        elif selection=='m vs N at B':
            plt.figure(figsize=(6,4))
            plt.grid()
            plt.title('$m^*(N,B)$')
            plt.xlabel('# iters')
            plt.ylabel('$m^*$')
            plt.plot(N_list,Nb_list)  
            plt.plot(N_list,N_list,color='orange')
            plt.legend(['adaptive $N_b$','$N$'])

        elif selection=='opt sched at B':
            plt.figure(figsize=(6,4))
            plt.grid()
            plt.title('optimal strategies')
            plt.xlabel('# iters')
            plt.ylabel('$\{\delta_{k}^*\}_{k=0}$')
            plt.semilogy(np.arange(N_opt),opt_sched)  
            plt.semilogy(np.arange(N_opt_constant),opt_sched_const)
            plt.legend(['optimal adaptive schedule','optimal fixed schedule'])
            
        elif selection=='repartition adaptive':
            repartition = [conv_term,error_term]
            repartition.append(np.sum(repartition))
            sources = np.arange(len(repartition))

            print('percentages (adaptive): conv term <- '+str(conv_term/(conv_term+error_term))+' , error term <- '+str(error_term/(conv_term+error_term)))

            fig, ax = plt.subplots(figsize=(9,6))
            plt.grid(axis='y')
            barlist = plt.bar(sources,repartition)
            plt.title('Repartition (adaptive) of the accuracy for N = '+str(int(N_opt)))
            plt.xticks(sources, ('Convergence Term (adaptive)', 'Error Term (adaptive)','Worst-Case Accuracy (adaptive)'))
            barlist[0].set_color('orange')
            barlist[1].set_color('red')
            barlist[2].set_color('blue')
                
        elif selection=='repartition fixed':
            repartition = [conv_term_const,error_term_const]
            repartition.append(np.sum(repartition))
            sources = np.arange(len(repartition))

            print('percentages (fixed): conv term <- '+str(conv_term_const/(conv_term_const+error_term_const))+' , error term <- '+str(error_term_const/(conv_term_const+error_term_const)))

            fig, ax = plt.subplots(figsize=(9,6))
            plt.grid(axis='y')
            barlist = plt.bar(sources,repartition)
            plt.title('Repartition (fixed) of the accuracy for N = '+str(int(N_opt_constant)))
            plt.xticks(sources, ('Convergence Term (fixed)', 'Error Term (fixed)','Worst-Case Accuracy (fixed)'))
            barlist[0].set_color('orange')
            barlist[1].set_color('red')
            barlist[2].set_color('blue')
                
        elif selection=='check-up Nb at opt (lin)':
            
            Nb_opt = Nb_smooth[np.argmin(values)]
            print('best Nb at '+str(int(Nb_opt)))
            
            plt.figure(figsize=(6,4))
            plt.grid()
            plt.title('check-up $N_b$ analytic')
            plt.xlabel('Nb')
            plt.ylabel('criterion')
            plt.plot(Nb_smooth,values)
            plt.scatter([Nb_opt],[np.min(values)],color='yellow')
            plt.legend(['$\epsilon(N,B,Nb)$','$\epsilon^*(N,B)$'])
            
        elif selection=='check-up Nb at opt (log)':
            
            Nb_opt = Nb_smooth[np.argmin(values)]
            print('best Nb at '+str(int(Nb_opt)))
            
            plt.figure(figsize=(6,4))
            plt.grid()
            plt.title('check-up $N_b$ analytic')
            plt.xlabel('Nb')
            plt.ylabel('criterion')
            plt.semilogy(Nb_smooth,values)
            plt.scatter([Nb_opt],[np.min(values)],color='yellow')
            plt.legend(['$\epsilon(N,B,Nb)$','$\epsilon^*(N,B)$'])
    
    plt.show()

def mer(N,B,K,beta,kappa):
    return np.max([0,np.ceil((2*N - 1 - np.sqrt(1+(8*(B-N*K))/(np.log(1+np.sqrt(kappa))*beta)))/2)])

def mer_bis(N,B,K,beta,kappa):
    return np.min([N-1,np.ceil((2*N - 1 + np.sqrt(1+(8*(B-N*K))/(np.log(1+np.sqrt(kappa))*beta)))/2)])

def epsiloner(B,K,beta,main,kappa):
        
    # range N?
    N_list = np.arange(1,np.floor(B/K)+1)
    
    # Nb?
    Nb_list = np.array([mer(N,B,K,beta,kappa) for N in N_list])
    
    # epsilons?
    eps_list = main*(1+np.sqrt(kappa))**(-(N_list-1)) + \
                ((1+np.sqrt(kappa))**(-(N_list-1))*((1+np.sqrt(kappa))**(Nb_list)-1))/(np.sqrt(kappa)) + \
                np.exp(-(B-N_list*K)/(beta*(N_list-Nb_list)))*(N_list-Nb_list)*(1+np.sqrt(kappa))**((-(N_list-1)+Nb_list)/2)
    
    eps_constant_list = main*(1+np.sqrt(kappa))**(-(N_list-1)) + \
        ((1+np.sqrt(kappa))**(-(N_list-1))*((1+np.sqrt(kappa))**(N_list)-1))/(np.sqrt(kappa))*np.exp(-(B-N_list*K)/(beta*N_list))
    
    # opt N?
    N_opt = N_list[np.argmin(eps_list)]
    N_opt_constant = N_list[np.argmin(eps_constant_list)]
    
    print('best accuracy achieved (adaptive) : '+str('{0:1.2e}'.format(np.min(eps_list)))+' @ N = '+str(int(N_opt)))
    print('best accuracy achieved (fixed) : '+str('{0:1.2e}'.format(np.min(eps_constant_list)))+' @ N = '+str(int(N_opt_constant)))
    
    # verification on Nb at opt (at least that)
    Nb_low,Nb_high = mer(N_opt,B,K,beta,kappa),mer_bis(N_opt,B,K,beta,kappa)
    Nb_smooth = np.linspace(Nb_low,Nb_high,500)
    values = main*(1+np.sqrt(kappa))**(-(N_opt-1)) + ((1+np.sqrt(kappa))**(-(N_opt-1))*((1+np.sqrt(kappa))**(Nb_smooth) - 1))/np.sqrt(kappa) + \
                    (N_opt-Nb_smooth)*(1+np.sqrt(kappa))**((-(N_opt-1) + Nb_smooth)/2)*np.exp(-(B-N_opt*K)/(beta*(N_opt-Nb_smooth)))
    
    # optimal schedules
    opt_sched_const = np.ones(int(N_opt_constant))*np.exp(-(B-N_opt_constant*K)/(beta*N_opt_constant))
    opt_Nb = mer(N_opt,B,K,beta,kappa)
    opt_sched = np.concatenate((np.ones(int(opt_Nb)),(1+np.sqrt(kappa))**((N_opt+opt_Nb-1-2*np.arange(opt_Nb,N_opt))/2)*np.exp(-(B-N_opt*K)/(beta*(N_opt-opt_Nb)))),axis=0)
    
    conv_term,error_term = main*(1+np.sqrt(kappa))**(-(N_opt-1)) , np.min(eps_list) - main*(1+np.sqrt(kappa))**(-(N_opt-1))
    conv_term_const,error_term_const = main*(1+np.sqrt(kappa))**(-(N_opt_constant-1)),np.min(eps_constant_list) - main*(1+np.sqrt(kappa))**(-(N_opt_constant-1))
    
    selections_widget = widgets.SelectMultiple(
    options=['acc vs N at B','m vs N at B','opt sched at B','repartition adaptive','repartition fixed','check-up Nb at opt (log)','check-up Nb at opt (lin)'],
    value=['acc vs N at B','m vs N at B'],
    description='Plot Choices',
    disabled=False)
    
    interact(lambda selections: options(selections,N_list,Nb_list,eps_list,eps_constant_list,N_opt,N_opt_constant,opt_sched_const,opt_Nb,opt_sched,conv_term,error_term,conv_term_const,error_term_const,Nb_smooth,values),\
            selections = selections_widget)

interact(epsiloner,B=B_widget,K=K_widget,beta=beta_widget,main=main_widget,kappa=kappa_widget);

interactive(children=(IntSlider(value=1000, continuous_update=False, description='B :', max=10050, min=10, ste…